# Data cleaning

After an initial exploration, cleaning the data is the first step of any analysis. Each dataset may need to be cleaned in a different way. Here, we will look for 3 aspects:

- **Missing values**: if some cells in our dataframes are empty, we will try to understand why and possibly attempt to fill them with values that make sense.
- **Duplicates**: duplicate rows might be caused by problems in the database (be it data entry or data extraction), we might have to drop some of them.
- **Data types**: numbers should have numerical data types, so that we can operate with them (take averages, sum and substract them, etc.).

## Reading the data

We'll do it file by file, in the "classical approach", but we will also provide code for automatizing the loading of multiple files. Feel free to explore this code on your own, understand it and use it in your project :)

#### Classical approach

Reading file by file

In [2]:
import pandas as pd

path = '../data/'
orderlines = pd.read_csv(r'C:\Users\Anja Wittler\OneDrive\Dokumente\TG\WBS\bootcamp\Sec_3_Pandas\orderlines.csv')
orders = pd.read_csv(r'C:\Users\Anja Wittler\OneDrive\Dokumente\TG\WBS\bootcamp\Sec_3_Pandas\orders.csv')
brands = pd.read_csv(r'C:\Users\Anja Wittler\OneDrive\Dokumente\TG\WBS\bootcamp\Sec_3_Pandas\brands.csv')
products = pd.read_csv(r'C:\Users\Anja Wittler\OneDrive\Dokumente\TG\WBS\bootcamp\Sec_3_Pandas\products.csv')

df_list = [orderlines, orders, brands, products]
files = ['orderlines','orders','brands','products']

#### Automatical approach

Using `os` and a loop to read all the files from a directory. It's also possible to read only files with a certain extension (like `.csv`):

In [2]:
# import pandas as pd
# import os
# path = '../data/eniac/'
# path, dirs, files = next(os.walk(path))
# #print(files)

# # remove non-csv files
# for file in files:
#     if not file.endswith("csv"):
#         files.remove(file)

In [3]:
# # create empty list 
# df_list = []

# # append datasets to the list
# for file in files:
#     temp_df = pd.read_csv(path + file, sep=',')
#     df_list.append(temp_df)

# # show results
# products, orders, orderlines, brands = df_list[0], df_list[1], df_list[2], df_list[3]

## Missing values

We will define a function that checks how many missing values are in each column of a dataframe:

In [3]:
def check_missing_values(data): 
    print('Missing values:' + '\n' + str(data.isna().sum()))

Now, loop through the list of dataframes we created above and apply the `check_missing_values` function to each one of them:

In [4]:
for i in range(len(df_list)): 
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    print('\n')

File name: orderlines
Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64


File name: orders
Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64


File name: brands
Missing values:
short    0
long     0
dtype: int64


File name: products
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64




## Duplicates

We will do the same with the duplicates: create a function to detect them in each dataframe, and then iterate through all the dataframes.

In [5]:
def check_duplicates(data): 
    print('Duplicated rows: ', data.duplicated().sum())

In [6]:
for i in range(len(df_list)): 
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    check_duplicates(df_list[i])
    print('\n')

File name: orderlines
Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64
Duplicated rows:  0


File name: orders
Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64
Duplicated rows:  0


File name: brands
Missing values:
short    0
long     0
dtype: int64
Duplicated rows:  0


File name: products
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64
Duplicated rows:  8746




## Data types

Based on our initial exploration we need to find a way to deal with our missing values. 

Do we have other problems?

* Example: problems with the `price` column in the `orderlines` dataframe.

We will now print all the previous outputs, plus information about each dataframe's data types. This will give us a "big picture" of how to need to proceed for the cleaning of the dataframes:

In [10]:
for i in range(len(df_list)):     
    print(f'File name: {files[i]}')
    print('\n')
    check_missing_values(df_list[i])
    print('\n')
    check_duplicates(df_list[i])
    print('\n')
    df_list[i].info()
    print('\n', '\n')

File name: orderlines


Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64


Duplicated rows:  0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB

 

File name: orders


Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64


Duplicated rows:  0


<class 'pandas.core.frame.DataFrame'

Some aspects that do not make a lot of sense:

* **products**: 
    * `price` and `promo_price` are loaded as objects. They should be of type numerical (floats).
    * Quite a lot of missing values: 
        * `description`: 7 missing values. Maybe the description could be inferred from the product name.
        * `price`: the missing values could be filled from the `orderlines` dataset. But first we will need to clean it. 

    
* **orders**: 
    * `created_date` should have a date data type. Then, it would be a good quality check to see if the created date for `orders` mathces with the created dates for `orderlines`. 
    * `total_paid` has 5 missing values. We can start by removing them since it's such a small amount, and in the future we can come back to the issue and investigate why these values are missing.
    
    
* **orderlines**: 
    * `unit_price` has to be a float, something wrong there. 
    * `date` has to be transformed to a date data type. Then, as we said, check that it with matches with the `orders` dataset. 
    
    
* **brands**: looks fine. 


Where do we have to start? Since `orders` and `orderlines` seem very crucial to the analysis, we will start by checking that they match.

## Cleaning `orders`


We will start by removing the rows with a missing value in the `total_paid` column:

In [13]:
# exploring the problematic rows
orders.loc[orders['total_paid'].isna()]

,order_id,created_date,total_paid,state
127701,427314,2017-11-20 18:54:39,NaN,Pending
132013,431655,2017-11-22 12:15:24,NaN,Pending
147316,447411,2017-11-27 10:32:37,NaN,Pending
148833,448966,2017-11-27 18:54:15,NaN,Pending
149434,449596,2017-11-27 21:52:08,NaN,Pending


These orders have a state of "Pending", so we can confidently exclude them from our dataset. We are interested on the completed ones.

In [14]:
orders.dropna(inplace=True)

The next cleaning bit will be changing the data type of the `created_date` column to datetime:

In [15]:
# change date datatype
orders['created_date'] = pd.to_datetime(orders['created_date'])

We will store the clean dataset as a csv in our computer, so that we have it ready for use in the next step:

In [28]:
orders.to_csv(path + 'orders_cl.csv', index=False)

## Cleaning orderlines

First let's transform our date time. 

In [17]:
orderlines['date'] = pd.to_datetime(orderlines['date'])

Now let's change the data type of `unit_price` to numeric.

In [4]:
# uncomment the following line of code and read the error message it outputs:
orderlines['unit_price'] = pd.to_numeric(orderlines['unit_price'])

ValueError: Unable to parse string "1.137.99" at position 6

While trying to transform this column to numeric an error appears. From the error message, "Unable to parse string "1.137.99" at position 6", it seems that our dataset has some problems with the thousands separators: they were encoded as dots, and python only admits one dot per number: the _decimal_ separator!

There are a lot of different ways to solve this problem. The first thing I will do is to count how many dots appear for each number using string operations. If price has more than one dot, I will consider it corrupted. 

In [7]:
# we create a copy of the dataset
ol_temp = orderlines.copy()

# create a new column with the amount of dots in the unit_price column
ol_temp['dots'] = orderlines['unit_price'].str.count('\.')

# show the rows with more than one dot
ol_temp.query('dots > 1')

,id,id_order,product_id,product_quantity,sku,unit_price,date,dots
6,1119115,299544,0,1,APP1582,1.137.99,2017-01-01 01:17:21,2
11,1119126,299549,0,1,PAC0929,2.565.99,2017-01-01 02:07:42,2
15,1119131,299553,0,1,APP1854,3.278.99,2017-01-01 02:14:47,2
43,1119195,299582,0,1,PAC0961,2.616.99,2017-01-01 08:54:00,2
59,1119214,299596,0,1,PAC1599,2.873.99,2017-01-01 09:53:11,2
...,...,...,...,...,...,...,...,...
293862,1649999,452946,0,1,APP2075,2.999.00,2018-03-14 13:03:33,2
293887,1650045,527321,0,1,PAC2148,3.497.00,2018-03-14 13:10:15,2
293889,1650050,527324,0,1,PAC2117,3.075.00,2018-03-14 13:10:56,2
293911,1650088,527342,0,1,APP2492,1.329.00,2018-03-14 13:24:51,2


Our theory about the thousands separators is confirmed. How can we solve this problem? 

Our approach will be to remove all the dots for all the unit_prices, and then add a dot before the last 2 digits to all the rows. Then we will transform it into numeric values.

In [22]:
orderlines = orderlines.assign(unit_price_nd = orderlines['unit_price'].str.replace('\.','', regex=True))
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374


In [23]:
orderlines['digits'] = orderlines['unit_price_nd'].str[-2:]
orderlines['integers'] = orderlines['unit_price_nd'].str[:-2]
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,digits,integers
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899,99,18
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900,00,399
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405,05,474
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839,39,68
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374,74,23


In [24]:
orderlines['new_unit_price'] = orderlines['integers'] + '.' + orderlines['digits']
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,digits,integers,new_unit_price
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899,99,18,18.99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900,00,399,399.00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405,05,474,474.05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839,39,68,68.39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374,74,23,23.74


In [25]:
# last step is to transform it to numeric
orderlines['unit_price'] = pd.to_numeric(orderlines['new_unit_price'])
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
 7   unit_price_nd     293983 non-null  object        
 8   digits            293983 non-null  object        
 9   integers          293983 non-null  object        
 10  new_unit_price    293983 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(5)
memory usage: 24.7+ MB


In [29]:
orderlines.drop(['unit_price_nd','digits','integers','new_unit_price'], 
                axis=1, inplace=True)

KeyError: "['unit_price_nd' 'digits' 'integers' 'new_unit_price'] not found in axis"

We're ready to store the `orderlines_cl` csv now:

In [30]:
orderlines.to_csv(path + 'orderlines_cl.csv', index=False)

## Challenge: Cleaning products

Now it is time to clean the products dataset. Follow the instructions in the platform to clean the `products` in the dataset.

In [8]:
# change type of price and promo price to numeric
products['price'] = pd.to_numeric(products['price'])

ValueError: Unable to parse string "1.639.792" at position 665

In [9]:
prod_temp = products.copy()
prod_temp['dots'] = prod_temp['price'].str.count('\.')
prod_temp.query('dots > 1')

,sku,name,desc,price,promo_price,in_stock,type,dots
665,CRU0015-2,Crucial memory Mac 16GB (2x8GB) SO-DIMM DDR3 1...,RAM 16GB (2x8GB) 135V MacBook Pro iMac (2012/2...,1.639.792,1.629.894,1,1364,2.0
792,APP0672,Apple iPhone 5S 16GB Space Gray,New iPhone 5S 16G Libre (ME432Y / AB).,4.694.994,4.694.994,0,NaN,2.0
797,APP0673,Apple iPhone 5S 16GB Silver,New Free iPhone 5S 16GB (ME433Y / A).,4.090.042,4.090.042,0,NaN,2.0
827,PAC0339,NewerTech miniStack 4TB Hard Drive Mac,External Box Hard Drive Mac + 4TB.,2.199.791,2.199.901,0,11935397,2.0
885,PAC0376,OWC Mercury Elite Pro Dual Thunderbolt + 8TB,RAID outer box 35 inch SATA connection Thunder...,5.609.698,5.549.895,0,11935397,2.0
...,...,...,...,...,...,...,...,...
19312,REP0424,Input repair Headphones iPad,Repair service including parts and labor for iPad,6.999.003,69.99,0,"1,44E+11",2.0
19313,REP0421,iPad charging connector repair,Repair service including parts and labor for iPad,6.999.003,69.99,0,"1,44E+11",2.0
19314,REP0416,iPad front camera repair,Repair service including parts and labor for iPad,6.999.003,69.99,0,"1,44E+11",2.0
19315,REP0413,repair rear camera iPad,Repair service including parts and labor for iPad,6.999.003,69.99,0,"1,44E+11",2.0


In [12]:
products = products.assign(price_nd = products['price'].str.replace('\.','', regex=True))
products.head()

,sku,name,desc,price,promo_price,in_stock,type,price_nd
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,5999
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401,59
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387,59
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230,25
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364,3499


In [13]:
products['digits'] = products['price_nd'].str[-2:]
products['integers'] = products['price_nd'].str[:-2]
products.head()

,sku,name,desc,price,promo_price,in_stock,type,price_nd,digits,integers
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,5999,99,59
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401,59,59,
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387,59,59,
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230,25,25,
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364,3499,99,34


In [14]:
products['new_price'] = products['integers'] + '.' + products['digits']
products.head()

,sku,name,desc,price,promo_price,in_stock,type,price_nd,digits,integers,new_price
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,5999,99,59,59.99
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401,59,59,,.59
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387,59,59,,.59
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230,25,25,,.25
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364,3499,99,34,34.99


In [15]:
products['price'] = pd.to_numeric(products['new_price'])
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          19326 non-null  object 
 1   name         19326 non-null  object 
 2   desc         19319 non-null  object 
 3   price        19280 non-null  float64
 4   promo_price  19326 non-null  object 
 5   in_stock     19326 non-null  int64  
 6   type         19276 non-null  object 
 7   price_nd     19280 non-null  object 
 8   digits       19280 non-null  object 
 9   integers     19280 non-null  object 
 10  new_price    19280 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 1.6+ MB


In [16]:
products.drop(['price_nd','digits','integers','new_price'], 
                axis=1, inplace=True)

In [ ]:
#products.to_csv(path + 'products_cl.csv', index=False)

In [17]:
products = products.assign(promo_price_nd = products['promo_price'].str.replace('\.','', regex=True))
products['digits'] = products['promo_price_nd'].str[-2:]
products['integers'] = products['promo_price_nd'].str[:-2]
products['new_promo_price'] = products['integers'] + '.' + products['digits']
products['promo_price'] = pd.to_numeric(products['new_promo_price'])
products.drop(['promo_price_nd','digits','integers','new_promo_price'], 
                axis=1, inplace=True)
products.head()

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,4998.99,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,0.59,5899.96,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,0.59,5698.98,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,0.25,2299.97,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364


In [ ]:
#products.to_csv(path + 'products_cl.csv', index=False)

In [20]:
products.loc[products['desc'].isna()]

,sku,name,desc,price,promo_price,in_stock,type
16126,WDT0211-A,"Open - Purple 2TB WD 35 ""PC Security Mac hard ...",NaN,1.07,8146.59,0,1298
16128,APP1622-A,Open - Apple Smart Keyboard Pro Keyboard Folio...,NaN,15682.06,15682.06,0,1298
17843,PAC2334,Synology DS718 + NAS Server | 10GB RAM,NaN,566.35,56598.96,0,12175397
18152,KAN0034-A,Open - Kanex USB-C Gigabit Ethernet Adapter Ma...,NaN,29.99,2379.25,0,1298
18490,HTE0025,Hyper Pearl 1600mAh battery Mini USB Mirror an...,NaN,24.99,22.99,1,1515
18612,OTT0200,OtterBox External Battery Power Pack 20000 mAHr,NaN,79.99,56.99,1,1515
18690,HOW0001-A,Open - Honeywell thermostat Lyric zonificador ...,NaN,199.99,14411.74,0,11905404


In [ ]:
#df.at['C', 'x'] = 10
products.at['16126', 'desc'] = 'Purple 2TB WD '